In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from IPython.display import Audio
import random
import whisper
import json
import librosa
import plotly.graph_objects as go
from data_loader import *
from utils import *
import openpyxl
import os
from sklearn.model_selection import train_test_split

In [3]:
# dataset_manager_first = SoundDatasetFirstTime("../Dataset")
# df_train, df_val = dataset_manager_first.load_datasets()
# print(f"Train dataset: {df_train.shape} samples")
# print(f"Validation dataset: {df_val.shape} samples")

In [4]:
dataset_manager = SoundDataset("../Dataset")
df_train, df_val, df_test = dataset_manager.load_datasets()
print(f"Train dataset: {df_train.shape} samples")
print(f"Validation dataset: {df_val.shape} samples")
print(f"Test dataset: {df_test.shape} samples")
df_train.head()

Train dataset: (11900, 5) samples
Validation dataset: (3400, 5) samples
Test dataset: (1700, 5) samples


,wav_id,label,audio_path,duration,text
0,1_3230,0,../Dataset/all/1_3230.wav,3.36,in regards to epoxy glue's lock tight has bee...
1,1_6231,2,../Dataset/all/1_6231.wav,2.97,great product ship to me fast it was as adver...
2,1_13194,2,../Dataset/all/1_13194.wav,2.92,this is a good value for the number of enduct...
3,1_14994,2,../Dataset/all/1_14994.wav,2.60,does a great job of cleaning all of the breasts?
4,2_849,2,../Dataset/all/2_849.wav,2.98,good for printing may be difficult to justify...


In [ ]:
df_train = df_train[['wav_id', 'label', 'audio_path', 'duration', 'word_count', 'text']]
df_val = df_val[['wav_id', 'label', 'audio_path', 'duration', 'word_count', 'text']]
df_test = df_test[['wav_id', 'label', 'audio_path', 'duration', 'word_count', 'text']]

# Display the first few rows to verify the change
df_train.head()

,wav_id,label,audio_path,duration,word_count,text
3230,1_3230,0,../Dataset/all/1_3230.wav,3.36,11.0,in regards to epoxy glue's lock tight has bee...
6231,1_6231,2,../Dataset/all/1_6231.wav,2.97,10.0,great product ship to me fast it was as adver...
13194,1_13194,2,../Dataset/all/1_13194.wav,2.92,11.0,this is a good value for the number of enduct...
14994,1_14994,2,../Dataset/all/1_14994.wav,2.60,10.0,does a great job of cleaning all of the breasts?
15849,2_849,2,../Dataset/all/2_849.wav,2.98,NaN,good for printing may be difficult to justify...


In [16]:
# Plots.plt_freq_labels(df_train, df_val)

### Translating audio into text

In [ ]:
import torch
print(torch.cuda.is_available())
!nvidia-smi

In [ ]:
# train_stats = calculate_duration_statistics(df_train)
# val_stats = calculate_duration_statistics(df_val)
# test_stats = calculate_duration_statistics(df_test)

# print(f"Train dataset - Mean: {train_stats[0]}, Median: {train_stats[1]}, Min: {train_stats[2]}, Max: {train_stats[3]}")
# print(f"Validation dataset - Mean: {val_stats[0]}, Median: {val_stats[1]}, Min: {val_stats[2]}, Max: {val_stats[3]}")
# print(f"Test dataset - Mean: {test_stats[0]}, Median: {test_stats[1]}, Min: {test_stats[2]}, Max: {test_stats[3]}")

# # Plots.plot_duration(df_train, df_val, df_test)

Validation dataset - Mean: 2.87, Median: 2.81, Min: 2.0, Max: 6.25


If the audio has a duration of less than 2 seconds, use Librosa to slow it down and save it
--> too difficult to understand both in the image and to transcribed

In [ ]:
## SLOW down the audio files that are <2sec
# slow_down_df(df_train)
# slow_down_df(df_test)
# slow_down_df(df_val)

Total files slowed down: 0


Denoise the audio file

In [ ]:
# ### DENOISE
# ### MAYBE MORE TIMES (?????)
# denoise_df(df_test)
# denoise_df(df_train)
# denoise_df(df_val)

KeyboardInterrupt: 

In [12]:
data_get_info(df_train, "1_3230")

Audio 1_3230 	Label for the audio: 0 
	Text:  in regards to epoxy glue's lock tight has been a disappointment.


Now we are ready to extract the text

In [ ]:
audio_manager = Audio2Text()

# df_train = audio_manager.transcribe_audio_files(df=df_train, batch_size=10, checkpoint_file = "text_test_denoised_ckpt.json", name="train")
# df_val = audio_manager.transcribe_audio_files(df=df_val, batch_size=10, checkpoint_file = "text_val_denoised_ckpt.json", name="val")
# df_test = audio_manager.transcribe_audio_files(df=df_test, batch_size=10, checkpoint_file = "text_test_denoised_ckpt.json", name="test")

c:\Users\alice\AppData\Local\pypoetry\Cache\virtualenvs\soundsentiment-HVhUS5V1-py3.10\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

	No previous checkpoint found, starting from scratch.
Processing batch:
  Processing  ../Dataset/all/1_1856.wav
  Processing  ../Dataset/all/1_1926.wav
  Processing  ../Dataset/all/1_1997.wav
  Processing  ../Dataset/all/1_14480.wav
  Processing  ../Dataset/all/1_4414.wav
  Processing  ../Dataset/all/2_879.wav
  Processing  ../Dataset/all/1_10942.wav
  Processing  ../Dataset/all/1_449.wav
  Processing  ../Dataset/all/1_9399.wav
  Processing  ../Dataset/all/1_11298.wav
	Checkpoint saved! It contains 10 transcriptions so far.
Processing batch:
  Processing  ../Dataset/all/1_10010.wav
  Processing  ../Dataset/all/2_1891.wav
  Processing  ../Dataset/all/1_2004.wav
  Processing  ../Dataset/all/1_6521.wav
  Processing  ../Dataset/all/1_13249.wav
  Processing  ../Dataset/all/1_8080.wav
  Processing  ../Dataset/all/1_4544.wav
  Processing  ../Dataset/all/1_3712.wav
  Processing  ../Dataset/all/1_11138.wav
  Processing  ../Dataset/all/1_3923.wav
	Checkpoint saved! It contains 20 transcriptions 

In [18]:
dataset_manager.save_dataset(df_train, "train")
dataset_manager.save_dataset(df_val, "val")
dataset_manager.save_dataset(df_test, "test")

Dataset saved as train.xlsx
Dataset saved as val.xlsx
Dataset saved as test.xlsx


In [19]:
df_val = process_text(df_val)
Plots.plot_count_words(df_val)

                           text   wav_id
59           güzel is injected.   1_3947
221              ез mini zaliki  1_12495
260        great result inside.      2_0
304    u affirmidesachme marrow   1_2807
440             well, me, okay.   1_2774
...                         ...      ...
3176          good night, baba.   1_4572
3179      info costume edition.   2_1588
3227        not desire enabled.  1_10610
3238             or is参, happy.   2_1666
3352             how you doing?  1_13316

[65 rows x 2 columns]
Max word count: 240
Average word count: 8.36
